<h1> Reconocimiento de Imágenes en CIFAR10 <h1>

**4.0 Importaciones necesarias**

Se importan librerías y funciones necesarias para trabajar en este problema. Se usará Theano como backend de la librería Keras.

In [1]:
from scipy.misc import imread
import cPickle as pickle
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from top_level_features import hog_features
from top_level_features import color_histogram_hsv
from top_level_features import extract_features

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce 820M (CNMeM is disabled, cuDNN not available)


**4.1 Extracción de archivos**

A continuación, se extraen los bloques correspondientes al conjunto de entrenamiento y al conjunto de pruebas, por medio de la función *unpickle*, que retorna los datos en formato de diccionario.

In [2]:
#Función para extracción de bloques de datos
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo)
    fo.close()
    return dict

**4.2 Construcción de conjuntos de entrenamiento, prueba y validación**

Por medio de la función *load_CIFAR_One*, es posible separar, para un determinado bloque, los atributos predictores y el atributo a predecir.

In [3]:
#Separación de atributos predictores y atributo a predecir
def load_CIFAR_one(filename):
    datadict = unpickle(filename)
    X = datadict['data']
    Y = datadict['labels']
    return X, np.array(Y)

A partir de la función anterior, se procede a construir matricialmente los conjuntos de entrenamiento, prueba y validación con que se trabajará más adelante. Para ello, se implementa, la función *load_CIFAR10*. Notar que el conjunto de validación se crea seleccionando las últimas cinco mil filas del conjunto de entrenamiento.

In [4]:
#Se crean, matricialmente, conjuntos de entrenamiento, prueba y validación
def load_CIFAR10(PATH):
    #Creación de matrices de entrenamiento
    xs = []
    ys = []
    for b in range(1, 6):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_one(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    
    #Creación de matrices de prueba
    Xts, Yts = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
    
    #Creación de matrices de validación
    #Se fusionan los conjuntos Xtr e Ytr y luego se permutan sus filas de forma aleatoria
    training_data = Xtr
    training_data_y = Ytr.reshape((50000, 1))
    training_data = np.append(training_data, training_data_y, axis=1)

    #Luego, se extrae el conjunto de validación a partir del conjunto de entrenamiento
    Xtr = training_data[:45000, :3072]
    Ytr = training_data[:45000, 3072]
    Xv = training_data[45000:, :3072]
    Yv = training_data[45000:, 3072]
    
    return Xtr, Ytr, Xts, Yts, Xv, Yv

De esta manera, se crean las matrices Xtr e Ytr, que corresponden a las imágenes y labels de entrenamiento, Xts e Yts, similes de los anteriores para el caso de pruebas y Xv e Yv, similes de los anteriores para el caso de validación.

In [5]:
Xtr, Ytr, Xts, Yts, Xv, Yv = load_CIFAR10('.')

**4.3 Normalización de datos**

Se estudian dos estrategias de normalización: Primero, se experimenta dividiendo los valores de las matrices por la intensidad máxima de pixel de cada una. Se comprueba que para todas estas matrices, dicha intensidad es 255. 

In [6]:
print ('Intesidad máxima de pixel')
print('')
print 'Conjunto de entrenamiento:', np.amax(Xtr)
print 'Conjunto de pruebas:', np.amax(Xts)
print 'Conjunto de validacion:', np.amax(Xv) 

Intesidad máxima de pixel

Conjunto de entrenamiento: 255
Conjunto de pruebas: 255
Conjunto de validacion: 255


Por tanto, se procede a dividir cada celda de las matrices Xtr, Xts y Xv por 255. Como consecuencia de esta operación, cada una de esta celdas posee un valor entre 0 y 1.

In [7]:
Xtr_px = Xtr / float(np.amax(Xtr)) 
Xts_px = Xts / float(np.amax(Xts))
Xv_px = Xv / float(np.amax(Xv))

La segunda estrategia de normalización consiste en escalar los datos de tal forma de que estos sigan una distribución gaussiana de media 0 y varianza 1. Para ello, se construye la función *scaler_function*.

In [8]:
def scaler_function(Xtr, Xts, Xv, scale=True):
    scaler_tr = StandardScaler(with_std=scale).fit(Xtr)
    scaler_ts = StandardScaler(with_std=scale).fit(Xts)
    scaler_v = StandardScaler(with_std=scale).fit(Xv)
    Xtr_scaled = scaler_tr.transform(Xtr)
    Xts_scaled = scaler_ts.transform(Xts)
    Xv_scaled = scaler_v.transform(Xv)
    return Xtr_scaled, Xts_scaled, Xv_scaled

Luego, bajo esta estrategia, se escalan las tres matrices originales.

In [9]:
Xtr_scaled, Xts_scaled, Xv_scaled = scaler_function(Xtr, Xts, Xv)

/home/seba/.virtualenvs/tareas-ml/local/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/seba/.virtualenvs/tareas-ml/local/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Sin embargo, es necesario precisar que la media de las matrices son muy cercanas a 0, pero no completamente 0, tal como se muestra a continuación:

In [10]:
Xtr_scaled.mean(axis=0)

array([ -2.83190755e-16,  -1.90901616e-16,   4.72338218e-17, ...,
         1.74231000e-17,  -2.00481607e-17,   1.44319125e-16])

In [11]:
Xts_scaled.mean(axis=0)

array([ -1.80144788e-16,   3.67983422e-17,  -1.17683641e-16, ...,
         7.66442465e-17,   6.45594689e-17,   7.08322290e-18])

In [12]:
Xv_scaled.mean(axis=0)

array([  2.23288055e-16,  -4.14335233e-17,  -7.75823850e-17, ...,
         1.57651669e-17,  -4.09894341e-17,   5.15587573e-17])

Por otro lado, las varianzas son exactamente 1: 

In [13]:
Xtr_scaled.std(axis=0)

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [14]:
Xts_scaled.std(axis=0)

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [15]:
Xv_scaled.std(axis=0)

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

**4.4 Construcción de redes neuronales para la resolución del problema planteado**

En esta sección, se implementan diferentes arquitecturas con el fin de encontrar el modelo que logre la mayor precisión de clasificación posible en el contexto del problema planteado. Se espera poder encontrar, a lo menos, un modelo cuya precisión sobre el conjunto de validación sea mayor o igual a 50%. Las pruebas serán llevadas a cabo en un equipo con procesador Intel Core i5 4030U y 8 GB de memoria. Al no disponer de una GPU adecuada para la ejecución de los experimentos, sólo se utilizará CPU para el entrenamiento, validación y testeo de las redes.

En primer lugar, se transforman los vectores Ytr, Yts e Yv a matrices categóricas. De esta forma, cada imágen tendrá asociada un vector binario para indicar la clase a la que pertenece. Por ejemplo, si una imágen pertenece a la clase 3, su vector binario será (0, 0, 1, 0, 0, 0, 0, 0, 0, 0). La transformación se lleva a cabo a continuación:

In [16]:
Ytr = to_categorical(Ytr)
Yts = to_categorical(Yts)
Yv = to_categorical(Yv)

**4.4.1 Implementación de red *feed forward* con una capa oculta y función de activación ReLU**

La primera arquitectura con que se trabaja es una red *feedforward*, la que se caracteriza por procesar datos de forma secuencial a través de sus capas. En este caso, se crea una red con una 1 capa oculta, compuesta por 50 neuronas. Además, se utiliza la función de activación de capas *ReLU*. Notar que se ha ajustado el parámetro *dropout* al 10%. Esto quiere decir que durante la fase de entrenamiento se desactivarán el 10% de las neuronas presentes en la capa oculta, siendo estas escogidas de forma totalmente aleatoria. Con esto, se busca evitar que el modelo se sobreajuste.

A continuación, se agrega la capa de salida, compuesta por 10 neuronas y utilizando la función de activación softmax. Luego, se compila el modelo. Una vez compilado, se procede a entrenarlo, al mismo tiempo que es optimizado. Es decir, en base a los valores que se obtienen para la función de perdida y para la precisión del modelo aplicado sobre el conjunto de validación, se realizan ajustes en los pesos asociados a cada neurona. Se usa gradiente descendente estocástico con tasa de aprendizaje 0.05 como método de optimización, llevando a cabo, en total, 50 iteraciones. Se consideran batches de tamaño 32. Dado a que las clases del problemas son mutuamente excluyentes, la función de perdida escogida es *categorical crossentropy*.

In [42]:
#Se crea modelo
model = Sequential()
#Se agrega capa oculta
model.add(Dense(50, kernel_initializer='uniform', input_dim=Xtr_scaled.shape[1], activation='relu'))
#Se ajusta parámetro dropout
model.add(Dropout(0.1))
#Se agrega capa de salida
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
#Se compila el modelo
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
#Se entrena el modelo
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
#Se evalúa el modelo sobre el conjunto de validación
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 7s - loss: 1.7905 - acc: 0.3598 - val_loss: 1.7989 - val_acc: 0.3668

Se puede observar que este modelo obtiene un pobre rendimiento sobre el conjunto de validación, alcanzando sólo un 45,48% de precisión.

In [44]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precisión sobre conjunto de validación: 45.48 %


Los resultados del código anterior muestran que cada iteración de optimización demora entre 6 y 10 segundos, sumando un tiempo total de 354 [s] para el total de iteraciones.

**4.4.2 Implementación de red *feed forward* con una capa oculta y función de activación tanh**

Dados los malos resultados del modelo de la sección anterior, se implementa una nueva red, cambiando la función de activación de neuronas en la capa oculta: Ahora se utiliza *tanh*. La arquitectura base se mantiene: Una capa oculta de 50 neuronas y una capa de salida de 10 neuronas. El parámetro *dropout* se mantiene en un 10%. También se mantienen el método de optimización usado, junto con la cantidad de iteraciones y el tamaño de los batches. 

In [45]:
model = Sequential()
model.add(Dense(50, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 7s - loss: 1.8101 - acc: 0.3595 - val_loss: 1.7426 - val_acc: 0.3938

Al usar la función de activación *tanh*, la precisión sobre el conjunto de validación empeora respecto a *ReLU* (43,26%), por lo que el rendimiento vuelve a ser pobre.

In [46]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precisión sobre conjunto de validación: 43.26 %


Sin embargo, cada iteración del proceso de optimización del modelo tuvo una duración de entre 4 y 8 segundos, siendo el tiempo total de 253 [s], por lo que se requiere de un tiempo bastante menor para entrenar el modelo definido con esta función de activación, en comparación con la arquitectura anterior.

**4.4.3 Aumentado la cantidad de neuronas en la capa oculta**

Dado a que la arquitectura implementada en la sección 4.4.1 obtuvo mejores resultados, se estudia el efecto de aumentar la cantidad de neuronas presentes en su capa oculta al doble, esto es, a 100 neuronas, manteniendo la función de activación.

In [47]:
model = Sequential()
model.add(Dense(100, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 5s - loss: 1.7816 - acc: 0.3663 - val_loss: 1.8306 - val_acc: 0.3666

Se puede ver que la precisión sobre el conjunto de validación es de un 47,2%, presentado un aumento de un 1,72% con respecto al primer modelo. Sin embargo, la precisión alcanzada sigue sin ser satisfactoria.

In [48]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precisión sobre conjunto de validación: 47.2 %


Por otro lado, cada iteración tuvo una duración de entre 5 y 6 segundos, siendo el tiempo total de entrenamiento de aproximadamente 251 [s], muy inferior al alcanzado por la arquitectura de la sección 4.4.1.

**4.4.4 Aumentando la cantidad de capas ocultas**

Se desea estudiar el efecto de agregar una segunda capa oculta a la arquitectura definida en la sección 4.4.3. Dicha capa posee las mismas características que la que creó previamente, en términos de cantidad de neuronas, función de activación y regularización.

In [18]:
model = Sequential()
model.add(Dense(100, input_dim=Xtr_scaled.shape[1], kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
# Se agrega segunda capa oculta
model.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(Xtr_scaled, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(Xv_scaled, Yv))
scores = model.evaluate(Xv_scaled, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 9s - loss: 1.7601 - acc: 0.3694 - val_loss: 1.5884 - val_acc: 0.4360
Epoch 2/50
45000/45000 [==============================] - 10s - loss: 1.5677 - acc: 0.4409 - val_loss: 1.5154 - val_acc: 0.4624
Epoch 3/50
45000/45000 [==============================] - 11s - loss: 1.4906 - acc: 0.4703 - val_loss: 1.5532 - val_acc: 0.4546
Epoch 4/50
45000/45000 [==============================] - 12s - loss: 1.4387 - acc: 0.4871 - val_loss: 1.4330 - val_acc: 0.4950
Epoch 5/50
45000/45000 [==============================] - 12s - loss: 1.4031 - acc: 0.4994 - val_loss: 1.4567 - val_acc: 0.4736
Epoch 6/50
45000/45000 [==============================] - 12s - loss: 1.3741 - acc: 0.5124 - val_loss: 1.4165 - val_acc: 0.4912
Epoch 7/50
45000/45000 [==============================] - 11s - loss: 1.3507 - acc: 0.5186 - val_loss: 1.4844 - val_acc: 0.4588
Epoch 8/50
45000/45000 [==============================] 

La precisión alcanzada por esta nueva arquitectura es de un 49,02%, siendo superior en un 1,82 % a la de la arquitectura implementada en la sección anterior. Sin embargo, tampoco se logra cumplir el objetivo planteado (precisión mayor o igual a 50% en el conjunto de validación).

In [21]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precision sobre conjunto de validacion: 49.02 %


Además, cada iteración del proceso de optimización tuvo una duración de entre 9 y 12 segundos, siendo el tiempo total de entrenamiento de 584 [s], es decir, más del doble del tiempo alcanzando en la sección 4.4.4.

**4.4.5 Implementación de red convolucional**

Ante los malos resultados conseguidos en las secciones anteriores, se desea estudiar el efecto de construir el clasificador de imágenes a partir del entrenamiento de una red convolucional, arquitectura muy popular, de acuerdo a la literatura, al momento de resolver este tipo de problemas, pues las arquitecturas convolucionales asumen que el input es una imágenes, por lo que están diseñadas especialmente para afontar problemas de este tipo. Para poder implementar de forma adecuada esta red, es necesario redimensionar previamente los conjuntos de entrenamiento, validación y prueba en base a la cantidad de imágenes pertenecientes a cada conjunto, las dimensiones de las imágenes (32 filas x 32 columnas) y la cantidad de canales asociados a cada uno (3 canales, pues se está trabajando con imágenes RGB).

In [22]:
#Dimensiones de imágenes de entrada
img_rows, img_cols = 32, 32
# Las imágenes están en formato RGB, es decir, se componen de 3 canales
img_channels = 3

#Se redimensionan conjuntos de entrenamiento, validación y pruebas
Xtr_scaled_conv = Xtr_scaled
Xv_scaled_conv = Xv_scaled
Xts_scaled_conv = Xts_scaled
Xtr_scaled_conv = Xtr_scaled_conv.reshape((Xtr_scaled.shape[0], img_rows, img_cols, img_channels))
Xv_scaled_conv = Xv_scaled_conv.reshape((Xv_scaled.shape[0], img_rows, img_cols, img_channels))
Xts_scaled_conv = Xts_scaled_conv.reshape((Xts_scaled.shape[0], img_rows, img_cols, img_channels))

Una vez redimensionados los conjuntos, se procede a implementar la red convolucional. Esta estará constituida por:

- Dos capas convolucionales 2D de 32 filtros, donde cada uno de ellos posee ancho y altura 3. Su función de activación es ReLU. Se desactivan el 25% de las neuronas.

- Una capa de reducción o *pooling*, encargada de reducir el tamaño del input recibido en un factor de 2 en cada dimensión.

- Dos capas convolucionales 2D de 64 filtros, donde cada uno de ellos posee ancho y altura 3. Su función de activación es ReLU. Se desactivan el 25% de las neuronas. 

- Una capa de reducción o *pooling*, igual a la añadida previamente.

- Una capa de tipo *flatten*, encargada de crear el vector de características a ser usado por la siguiente capa.

- Una capa densa de 512 neuronas, con función de activación ReLU. Se desactivan el 50% de las neuronas.

- La capa de salida, compuesta por 10 neuronas. Su función de activación es softmax.

In [25]:
# Se crea modelo
model = Sequential()

# Se añaden dos capas convolucionales de 32 filtros y una capa de pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=Xtr_scaled_conv.shape[1:], activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Se añaden dos capas convolucionales de 64 filtros y una capa de pooling
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Se genera vector de características
model.add(Flatten())

# Se añade capa densa
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Se añade capa de salida
model.add(Dense(10, activation='softmax'))

Ya definido el modelo, se procede a compilar, entrenar y evaluar el modelo en condiciones similares a las de las secciones anteriores, excepto por la cantidad de iteraciones, que se ha fijado en 15, dado a que el tiempo requerido para entrenar una red convolucional en CPU puede llegar a ser extremadamente alto.

In [26]:
#Se compila el modelo
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05), metrics=['accuracy'])

# Se entrena el modelo
model.fit(Xtr_scaled_conv, Ytr, batch_size=32, epochs=15, verbose=1, validation_data=(Xv_scaled_conv, Yv))

# Se evalúa el modelo
scores = model.evaluate(Xv_scaled_conv, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/15
45000/45000 [==============================] - 379s - loss: 1.7709 - acc: 0.3554 - val_loss: 1.6414 - val_acc: 0.4288
Epoch 2/15
45000/45000 [==============================] - 376s - loss: 1.4833 - acc: 0.4692 - val_loss: 1.3224 - val_acc: 0.5250
Epoch 3/15
45000/45000 [==============================] - 378s - loss: 1.3602 - acc: 0.5163 - val_loss: 1.2593 - val_acc: 0.5538
Epoch 4/15
45000/45000 [==============================] - 378s - loss: 1.2761 - acc: 0.5483 - val_loss: 1.2475 - val_acc: 0.5658
Epoch 5/15
45000/45000 [==============================] - 379s - loss: 1.2067 - acc: 0.5736 - val_loss: 1.3099 - val_acc: 0.5406
Epoch 6/15
45000/45000 [==============================] - 379s - loss: 1.1508 - acc: 0.5938 - val_loss: 1.0915 - val_acc: 0.6230
Epoch 7/15
45000/45000 [==============================] - 384s - loss: 1.1080 - acc: 0.6105 - val_loss: 1.2466 - val_acc: 0.5528
Epoch 8/15
45000/45000 [========================

Se observa que la precisión alcanzada por la red convolucional sobre el conjunto de validación es de un 68,62%, cifra bastante superior en comparación con las precisiones anteriores, demostrando la conveniencia de la utilización de una arquitectura convolucional para atacar el problema propuesto. De hecho, lo más probable es que si se aumentara el número de iteraciones, sería posible conseguir una precisión aún mayor. Sin embargo, es necesario tener en cuenta que cada una de dichas iteraciones tiene asociado un alto tiempo de duración (entre 372 y 384 segundos; tiempo total de entrenamiento: 5699 [s] = 1 hora y 35 minutos, aproximadamente), por lo que puede llegar un punto en que el entrenamiento del modelo sea inviable en términos de tiempo (a menos que este sea entrenado en GPU).

In [27]:
print 'Precision sobre conjunto de validacion:', scores[1] * 100, '%'

Precision sobre conjunto de validacion: 68.62 %


En consecuencia, se escoge el modelo convolucional para ser evaluado sobre el conjunto de pruebas. 

In [28]:
scores_test = model.evaluate(Xts_scaled_conv, Yts)
print 'Precision sobre conjunto de pruebas:', scores_test[1] * 100, '%'

10000/10000 [==============================] - 36s    
Precision sobre conjunto de pruebas: 67.24 %


Así, se obtiene una precisión de un 67,24% sobre el conjunto de pruebas, lo que permite asegurar que el modelo convolucional construído es el más indicado para resolver el problema de clasificación planteado, aunque sólo en comparación con las arquitecturas anteriormente utilizadas, pues el modelo está lejos de ser el más robusto.

**4.5 Cambio en el formato de representación de imágenes**

En esta sección, se estudian dos nuevos formatos de representación de los datos: El primero, corresponde a un histograma de tonos. El segundo, a descriptores HOG. En el siguiente código, se crean estas dos representaciones, junto con una tercera representación que emplea las dos anteriores de forma simultánea. Con ello, se espera poder encontrar representaciones para las imágenes tales que permitan construir modelos más robustos.

In [17]:
# Se extraen características en conjunto de entrenamiento
features1_tr = extract_features(Xtr,[hog_features])  # extrae hog features
features2_tr = extract_features(Xtr,[color_histogram_hsv])  # extrae histogramas de tonos
features3_tr = extract_features(Xtr,[hog_features, color_histogram_hsv])  # extrae todo

# Se extraen características en conjunto de prueba
features1_ts = extract_features(Xts,[hog_features])
features2_ts = extract_features(Xts,[color_histogram_hsv])
features3_ts = extract_features(Xts,[hog_features, color_histogram_hsv]) 

# Se extraen características en conjunto de validación
features1_v = extract_features(Xv,[hog_features])
features2_v = extract_features(Xv,[color_histogram_hsv])
features3_v = extract_features(Xv,[hog_features, color_histogram_hsv])

(45000, 32, 32, 3)
(45000, 32, 32, 3)
(45000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(5000, 32, 32, 3)
(5000, 32, 32, 3)
(5000, 32, 32, 3)


Si bien se esperaba poder utilizar la arquitectura implementada en la sección 4.4.5 para implementar los nuevos clasificadores, ello no ha sido posible, pues las redes convolucionales asumen que los inputs tienen un formato definido, esto es, imágenes de 32 x 32 píxeles y 3 canales. Sin embargo, la función *extract_features*, encargada de crear las nuevas representaciones, deja a cada imágen como una matriz como un vector de 144 características, en el caso de los descriptores HOG, mientras que en el caso de los histogramas de tonos, tal vector está conformado por 10 características. En otras palabras, estas nuevas representaciones no son compatibles con el input esperado por una red convolucional, por lo que será necesario retomar alguna de las arquitecturas utilizadas en las secciones anteriores. Dado a que la arquitectura de la sección 4.4.4 fue la que consiguió la precisión más alta después de la red convolucional, será reutilizada en las siguientes secciones.

** 4.5.1 Entrenamiento vía descriptores HOG **

Se procede a entrenar el modelo de la sección 4.4.4 a partir de la representación vía descriptores HOG.

In [19]:
model_hog = Sequential()
model_hog.add(Dense(100, input_dim=features1_tr.shape[1], kernel_initializer='uniform', activation='relu'))
model_hog.add(Dropout(0.1))
model_hog.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model_hog.add(Dropout(0.1))
model_hog.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model_hog.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_hog.fit(features1_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features1_v, Yv))
scores = model_hog.evaluate(features1_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 3s - loss: 1.7122 - acc: 0.3877 - val_loss: 1.5090 - val_acc: 0.4770
Epoch 2/50
45000/45000 [==============================] - 3s - loss: 1.4810 - acc: 0.4839 - val_loss: 1.4481 - val_acc: 0.4836
Epoch 3/50
45000/45000 [==============================] - 3s - loss: 1.4000 - acc: 0.5110 - val_loss: 1.3587 - val_acc: 0.5224
Epoch 4/50
45000/45000 [==============================] - 3s - loss: 1.3499 - acc: 0.5256 - val_loss: 1.3776 - val_acc: 0.5154
Epoch 5/50
45000/45000 [==============================] - 3s - loss: 1.3137 - acc: 0.5411 - val_loss: 1.3277 - val_acc: 0.5336
Epoch 6/50
45000/45000 [==============================] - 3s - loss: 1.2847 - acc: 0.5503 - val_loss: 1.3110 - val_acc: 0.5402
Epoch 7/50
45000/45000 [==============================] - 3s - loss: 1.2585 - acc: 0.5553 - val_loss: 1.3329 - val_acc: 0.5294
Epoch 8/50
45000/45000 [==============================] - 3s -

Se puede ver como la precisión sobre el conjunto de validación es de un 54,44%, menor a la de la red convolucional, pero superior en un 5,42% a la precisión de la red entrenada sobre la representación original (sección 4.4.4).

In [21]:
print 'Precision sobre conjunto de validacion (Caso: Descriptores HOG):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Descriptores HOG): 54.44 %


Por otro lado, el tiempo de cómputo se reduce, siendo la duración de cada iteración de entre 3 y 4 segundos, sumando un tiempo total de 152 [s], muy inferior a los 584 [s] alcanzados sobre la representación original. 

**4.5.2 Entrenamiento vía histogramas de colores** 

Usando la representación mediante histogramas de tonos, se vuelve a entrenar el modelo.

In [22]:
model_col_hist = Sequential()
model_col_hist.add(Dense(100, input_dim=features2_tr.shape[1], kernel_initializer='uniform', activation='relu'))
model_col_hist.add(Dropout(0.1))
model_col_hist.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model_col_hist.add(Dropout(0.1))
model_col_hist.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model_col_hist.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_col_hist.fit(features2_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features2_v, Yv))
scores = model_col_hist.evaluate(features2_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 3s - loss: 2.3005 - acc: 0.1126 - val_loss: 2.2932 - val_acc: 0.1182
Epoch 2/50
45000/45000 [==============================] - 3s - loss: 2.1857 - acc: 0.1763 - val_loss: 2.1066 - val_acc: 0.2048
Epoch 3/50
45000/45000 [==============================] - 3s - loss: 2.1063 - acc: 0.1983 - val_loss: 2.0813 - val_acc: 0.2166
Epoch 4/50
45000/45000 [==============================] - 3s - loss: 2.0745 - acc: 0.2120 - val_loss: 2.0506 - val_acc: 0.2254
Epoch 5/50
45000/45000 [==============================] - 3s - loss: 2.0593 - acc: 0.2228 - val_loss: 2.0441 - val_acc: 0.2318
Epoch 6/50
45000/45000 [==============================] - 3s - loss: 2.0535 - acc: 0.2248 - val_loss: 2.0403 - val_acc: 0.2324
Epoch 7/50
45000/45000 [==============================] - 4s - loss: 2.0496 - acc: 0.2277 - val_loss: 2.0374 - val_acc: 0.2370
Epoch 8/50
45000/45000 [==============================] - 3s -

Analizando los resultados, se puede ver como la precisión sobre el conjunto de validación es muy pobre (27,18%), siendo totalmente inferior a cualquiera de las arquitecturas antes implementadas. Observando que la precisión sobre el conjunto de entrenamiento también es baja (26,56%), se está ante un modelo que sufre de *underfitting*, esto es, la arquitectura no es capaz de comprender la forma en que se relacionan las variables, lo que resulta razonable considerando que la representación utilizada cuenta con sólo 10 características, probablemente una cantidad insuficiente para construir un buen clasificador.

In [23]:
print 'Precision sobre conjunto de validacion (Caso: Histogramas de colores):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Histogramas de colores): 27.18 %


**4.5.3 Entrenamiento vía combinación de representaciones**

Finalmente, se usa la representación combinada de descriptores HOG e histogramas de color para entrenar el modelo.

In [24]:
model_comb = Sequential()
model_comb.add(Dense(100, input_dim=features3_tr.shape[1], kernel_initializer='uniform', activation='relu'))
model_comb.add(Dropout(0.1))
model_comb.add(Dense(100, kernel_initializer='uniform', activation='relu'))
model_comb.add(Dropout(0.1))
model_comb.add(Dense(10, kernel_initializer='uniform', activation='softmax'))
model_comb.compile(optimizer=SGD(lr=0.05), loss='categorical_crossentropy', metrics=['accuracy'])
model_comb.fit(features3_tr, Ytr, epochs=50, batch_size=32, verbose=1, validation_data=(features3_v, Yv))
scores = model_comb.evaluate(features3_v, Yv)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 3s - loss: 1.7141 - acc: 0.3862 - val_loss: 1.6198 - val_acc: 0.4106
Epoch 2/50
45000/45000 [==============================] - 3s - loss: 1.4677 - acc: 0.4850 - val_loss: 1.4649 - val_acc: 0.4874
Epoch 3/50
45000/45000 [==============================] - 3s - loss: 1.3813 - acc: 0.5180 - val_loss: 1.3369 - val_acc: 0.5326
Epoch 4/50
45000/45000 [==============================] - 3s - loss: 1.3170 - acc: 0.5386 - val_loss: 1.4153 - val_acc: 0.5120
Epoch 5/50
45000/45000 [==============================] - 3s - loss: 1.2804 - acc: 0.5503 - val_loss: 1.3126 - val_acc: 0.5324
Epoch 6/50
45000/45000 [==============================] - 3s - loss: 1.2460 - acc: 0.5619 - val_loss: 1.2855 - val_acc: 0.5538
Epoch 7/50
45000/45000 [==============================] - 3s - loss: 1.2211 - acc: 0.5688 - val_loss: 1.3201 - val_acc: 0.5392
Epoch 8/50
45000/45000 [==============================] - 3s -

Los resultados muestran que se alcanza una precisión de 57,24% sobre el conjunto de validación, superior en un 2,8% a la precisión de la arquitectura de la sección 4.5.1. e inferior en un 11,38% a la precisión de la red convolucional. 

In [25]:
print 'Precision sobre conjunto de validacion (Caso: Representaciones combinadas):', scores[1] * 100, '%'

Precision sobre conjunto de validacion (Caso: Representaciones combinadas): 57.24 %


En términos de tiempo, no existen diferencias con la arquitectura de la sección 4.5.1, siendo sus tiempos de entrenamiento iguales (152 [s]). 

En consecuencia, se elige este último modelo para ser evaluado sobre el conjunto de pruebas.

In [29]:
scores_test = model_comb.evaluate(features3_ts, Yts)
print ''
print 'Precision sobre conjunto de pruebas (Caso: Representaciones combinadas):', scores_test[1] * 100, '%'

 9728/10000 [============================>.] - ETA: 0s
Precision sobre conjunto de pruebas (Caso: Representaciones combinadas): 55.63 %


Se observa que la precisión alcanzada sobre el conjunto de pruebas es de un 55,63%, muy inferior a la alcanzada por la red convolucional (67,24%).

**4.6 Conclusiones**

Los modelos implementados en las secciones 4.4.1, 4.4.2, 4.4.3 y 4.4.4 alcanzan un pobre rendimiento sobre el conjunto de validación. Además, el rendimiento sobre el conjunto de entrenamiento fue superior en un 20% aproximadamente en cada uno de los casos. De esta situación se desprende que tales modelos están sobreajustados y por tanto son incapaces de generalizar el problema a partir de los datos de entrenamiento.

Por otro lado, la red convolucional construída en la sección 4.4.5 alcanzó una precisión más que aceptable, tanto sobre el conjunto de validación como de pruebas, siendo dichas precisiones muy cercanas a la obtenida sobre el conjunto de entrenamiento, lo que quiere decir que se redujo en parte el fenómeno de *overfitting*, probablemente gracias a la disminución de la cantidad de características asociadas a cada imágen. Ahora bien, los tiempos de ejecución aumentaron notablemente, pero se reitera que esto pueda evitarse si se cuenta con una GPU adecuada para realizar el entrenamiento.

Los modelos implementados en las secciones 4.5.1 y 4.5.3 alcanzaron mejores rendimientos respecto a los utilizados en las secciones 4.4.1 - 4.4.5 en términos de precisión y tiempo, además de presentar un sobreajuste menor (diferencias de aproximadamente 10% entre precisiones de validación y entrenamiento, en cada caso). Sin embargo, la precisión obtenida por la arquitectura 4.5.3 sobre el conjunto de pruebas es muy inferior a la de la red convolucional.

De esta manera, la red convolucional se identifica como la arquitectura más adecuada para resolver el problema, no obstante, se debe contar con los recursos computacionales adecuados para que su implementación pueda ser llevada a cabo en forma eficiente. En caso contrario, se debe utilizar una red *feedforward* y utilizar alguna representación que reduzca la cantidad de características asociadas a cada imágen, a costa de un bajo rendimiento sobre ejemplos nuevos.